## Семинар 1 Индекс

## Intro

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [0]:
### _check : в коллекции должно быть около 165 файлов

In [1]:
pip install pymorphy2

  Using cached https://files.pythonhosted.org/packages/a3/33/fff9675c68b5f6c63ec8c6e6ff57827dda28a1fa5b2c2d727dffff92dd47/pymorphy2-0.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/51/2465fd4f72328ab50877b54777764d928da8cb15b74e2680fc1bd8cb3173/pymorphy2_dicts-2.4.393442.3710985-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6a/84/ff1ce2071d4c650ec85745766c0047ccc3b5036f1d03559fd46bb38b5eeb/DAWG_Python-0.7.2-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


С помощью обратного индекса посчитайте:  


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный? 


In [2]:
import os
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [7]:
os.chdir("/Users/Asus/friends")

In [13]:
for file in os.listdir(os.getcwd()):
    print(file)

Friends - season 1
Friends - season 2
Friends - season 3
Friends - season 4
Friends - season 5
Friends - season 6
Friends - season 7


In [14]:
top = os.getcwd()
top

'C:\\Users\\Asus\\friends'

In [15]:
folder = []
for i in os.walk(top):
    folder.append(i)

In [19]:
allTexts = []
allNames = []
for address, dirs, files in folder:
    for file in files:
        allNames.append(file)
        with open(address+'/'+file, 'r', encoding = 'utf-8') as f:
            text = f.read()
            allTexts.append(text)

In [20]:
len(allTexts)

165

In [22]:
allWordsAllTexts = []
for text in allTexts:
    allWordsInOneText = []
    if text != "":
        for word in text.split():
            word = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', word).lower()
            if word != "":
                p = morph.parse(word)[0]
                if p not in allWordsInOneText:
                    allWordsInOneText.append(p.normal_form)
        allWordsAllTexts.append(allWordsInOneText)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [24]:
vectorizer = CountVectorizer()

In [25]:
arrayString = []
for text in allWordsAllTexts:
    s = ' '.join(text)
    arrayString.append(s)

In [26]:
X = vectorizer.fit_transform(arrayString)

In [27]:
Matrix = X.toarray()

In [28]:
print(Matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [29]:
import pandas as pd
df = pd.DataFrame(Matrix, index=allNames, columns=vectorizer.get_feature_names())

In [30]:
df.head()

,aa,ad,afr,aga,ahh,ar,ay,aнгел,bay,bhd,...,ящичек,ёй,ёкнуть,ёлка,ёлочный,ёпэрэсотэ,ёрл,ёрш,ёршик,ёще
Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x02 - The One With The Sonogram At The End.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x03 - The One With The Thumb.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x04 - The One With George Stephanopoulos.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x05 - The One With The East German Laundry Detergent.ru.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Поисковик**

In [31]:
from collections import Counter
import operator
import math

In [32]:
def tokenize(doc):
    allWordsInOneText = []
    text = doc.split()
    for word in text:
        word = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', word).lower()
        if word != "":
            p = morph.parse(word)[0]
            allWordsInOneText.append(p.normal_form)
    return allWordsInOneText

In [33]:
def build_terms(corpus):
    terms = {}
    current_index = 0
    for doc in corpus:
        for word in tokenize(doc):
            if word not in terms:
                terms[word] = current_index
                current_index += 1
    return terms

In [34]:
def tf(document, terms):
    words = tokenize(document)
    total_words = len(words)
    doc_counter = Counter(words)
    for word in doc_counter:
        doc_counter[word] /= total_words
    tfs = [0 for _ in range(len(terms))]
    for term, index in terms.items():
        tfs[index] = doc_counter[term]
    return tfs

In [35]:
def _count_docs_with_word(word, docs):
    counter = 1
    for doc in docs:
        if word in doc:
            counter += 1
    return counter

In [36]:
# documents - это корпус
def idf(documents, terms):
    idfs = [0 for _ in range(len(terms))]
    total_docs = len(documents)
    for word, index in terms.items():
        docs_with_word = _count_docs_with_word(word, documents)
        idf = 1 + math.log10(total_docs / docs_with_word)
        idfs[index] = idf
    return idfs

In [37]:
def _merge_td_idf(tf, idf, terms):
    return [tf[i] * idf[i] for i in range(len(terms))]


def build_tfidf(corpus, document, terms):
    doc_tf = tf(document, terms)
    doc_idf = idf(corpus, terms)
    return _merge_td_idf(doc_tf, doc_idf, terms)


def cosine_similarity(vec1, vec2):
    def dot_product2(v1, v2):
        return sum(map(operator.mul, v1, v2))

    def vector_cos5(v1, v2):
        prod = dot_product2(v1, v2)
        len1 = math.sqrt(dot_product2(v1, v1))
        len2 = math.sqrt(dot_product2(v2, v2))
        return prod / (len1 * len2)

    return vector_cos5(vec1, vec2)

In [38]:
tf_idf_total = []
corpus = allTexts
terms = build_terms(corpus)

In [39]:
len(corpus)

165

In [40]:
for document in corpus:
    tf_idf_total.append(build_tfidf(corpus, document, terms))

In [41]:
query = "Он подарил мне цветы"

In [42]:
results = {}
query_tfidf = build_tfidf(corpus, query, terms)
for index, document in enumerate(tf_idf_total):
    results[index] = cosine_similarity(query_tfidf, document)

In [43]:
ld = list(results.items())
ld.sort(key=lambda i: i[1], reverse= True)
ld

[(52, 0.2752542859743394),
 (75, 0.24471401179804134),
 (135, 0.23290323882225525),
 (51, 0.2329008795436449),
 (27, 0.22910673441624954),
 (138, 0.22879058779910527),
 (47, 0.22125968103179108),
 (55, 0.2197335939372848),
 (143, 0.2183933635782559),
 (2, 0.21734952102205848),
 (9, 0.2169524068135041),
 (145, 0.21692765155774396),
 (158, 0.21689218055271234),
 (40, 0.21662217414530624),
 (107, 0.21441365055359127),
 (98, 0.21388745853319852),
 (78, 0.2138853573391441),
 (41, 0.2130222236381641),
 (59, 0.21277829270285628),
 (33, 0.2119639089140637),
 (72, 0.21159810933109935),
 (90, 0.21144027786533223),
 (91, 0.2112619947559666),
 (162, 0.21015920308428201),
 (77, 0.20964690328532032),
 (19, 0.20959084004177658),
 (80, 0.2078521076832388),
 (136, 0.20774321164955095),
 (151, 0.20672401103432556),
 (26, 0.20671838812314927),
 (82, 0.20593766518551734),
 (73, 0.20283250185744015),
 (22, 0.20266241032202806),
 (30, 0.20254230232391018),
 (56, 0.20223281636772597),
 (63, 0.202170786910035

In [44]:
print('По вашему запросу лучший результат - ' + allNames[ld[0][0]])

По вашему запросу лучший результат - Friends - 3x08 - The One With The Giant Poking Device.ru.txt
